In [1]:
import pandas as pd
import numpy as np
import requests 
from bs4 import BeautifulSoup
from tqdm import tqdm
from selenium import webdriver
import time
import re
import os

In [63]:
def int_to_month(month):
    if month == 1:
        month = 'Январь'
    elif month == 2:
        month = 'Февраль'
    elif month == 3:
        month = 'Март'
    elif month == 4:
        month = 'Апрель'
    elif month == 5:
        month = 'Май'
    elif month == 6:
        month = 'Июнь'
    elif month == 7:
        month = 'Июль'
    elif month == 8:
        month = 'Август'
    elif month == 9:
        month = 'Сентябрь'
    elif month == 10:
        month = 'Октябрь'
    elif month == 11:
        month = 'Ноябрь'
    elif month == 12:
        month = 'Декабрь'
    return month

def splitdate(date):
    date_split = date.split('.')
    day = int(date_split[0])
    month = int(date_split[1])
    strmonth = int_to_month(month)
    if month == 1:
        month = 'января'
    elif month == 2:
        month = 'февраля'
    elif month == 3:
        month = 'марта'
    elif month == 4:
        month = 'апреля'
    elif month == 5:
        month = 'мая'
    elif month == 6:
        month = 'июня'
    elif month == 7:
        month = 'июля'
    elif month == 8:
        month = 'августа'
    elif month == 9:
        month = 'сентября'
    elif month == 10:
        month = 'октября'
    elif month == 11:
        month = 'ноября'
    elif month == 12:
        month = 'декабря'
    year = int(date_split[2])
    return day, month, year, strmonth

def get_result(score1, score2):
    if score1 > score2:
        result1 = 'Победа'
        result2 = 'Поражение'
    elif score2 > score1:
        result2 = 'Победа'
        result1 = 'Поражение'
    else:
        result1 = result2 = 'Ничья'
    return result1, result2

def fullrole(role):
    if role == 'ВР':
        return 'Вратарь'
    if role == 'ЗАЩ':
        return 'Защитник'
    if role == 'П/З':
        return 'Полузащитник'
    if role == 'НАП':
        return 'Нападающий'
    return np.nan

def get_idteam(team, teams_dict):
    id_team = teams_dict.get(team)
    if id_team is None:
        teams_dict['COUNTER'] += 1
        id_team = teams_dict['COUNTER']
        teams_dict[team] = id_team
    return id_team

def get_idplayer(player, players_dict):
    id_player = players_dict.get(player)
    if id_player is None:
        players_dict['COUNTER'] += 1
        id_player = players_dict['COUNTER']
        players_dict[player] = id_player
    return id_player

def get_chrono(match_soup):
    '''
    гол, ассист, пенальти, жк, кк, автогол
    '''
    chrono_dict = dict()
    chrono = match_soup.find('table', class_='chronology')
    events = chrono('tr')[:-1]
    for event in events:
        if event.a is None or event.a.string is None or event.i is None:
            continue
        kind = event.div.get('class')[0]
        name = event.a.string.strip()
        kinds = chrono_dict.get(name, [0, 0, 0, 0, 0, 0])
        if kind == 'event1':
            kinds[0] += 1
        elif kind == 'event1-10':
            kinds[2] += 1
        elif kind == 'event1-11':
            kinds[5] += 1
        elif kind == 'event3':
            kinds[3] += 1
        elif kind == 'event4':
            kinds[4] += 1
        elif kind == 'event4-3':
            kinds[3] += 1
            kinds[4] += 1
        chrono_dict[name] = kinds
    return chrono_dict

def get_playertime(player, main):
    if main == 'Старт':
        starttime = 0
        end = player.b
        if end and len(end.string.strip()) > 1:
            endtime = int(end.string.strip()[:-1])
        else:
            endtime = 90
    else:
        start = player.b
        if start and len(start.string.strip()) > 1:
            starttime = int(start.string.strip()[:-1])
            end = start.find_next_sibling('b')
            if end and len(end.string.strip()) > 1:
                endtime = int(end.string.strip()[:-1])
            else:
                endtime = 90
        else:
            starttime = np.nan
            endtime = np.nan
    playtime = endtime - starttime
    return starttime, endtime, playtime

def get_playerinfo(player, main, players_dict, chrono_dict):
    
    secondname = player.a.string
    
    player_link = premier + player.a.get('href')
    player_page = requests.get(player_link)
    player_soup = BeautifulSoup(player_page.text)
    
    fullname = player_soup.h1.string.strip()
    firstname = fullname.split()[1]
    
    id_player = get_idplayer(fullname, players_dict)
    
    content = player_soup.find('div', class_='player-content')('p')
    bd = content[0].span.find_next('span').contents[1][1:-3]
    bd_day, bd_month, bd_year, bd_strmonth = splitdate(bd)
    bd_date = str(bd_day) + ' ' + bd_month + ' ' + str(bd_year)
    height = content[2].span.find_next('span').contents[1][1:-4]
    weight = content[3].span.find_next('span').contents[1][1:-3]
    
    number_role = player.i.string.split()
    try:
        role = fullrole(number_role[1])
    except:
        role = np.nan
    try:
        number = int(number_role[0][1:-1])
    except:
        number = np.nan
    
    starttime, endtime, playtime = get_playertime(player, main)
    
    goals, assists, penalties, yc, rc, autogoals = chrono_dict.get(secondname, [0, 0, 0, 0, 0, 0])
    
    return (role, number, fullname, id_player, firstname, secondname, starttime, endtime, playtime,
            goals, assists, yc, rc, penalties, autogoals, player_link, bd_date, bd_strmonth, height, weight, main)

def get_stinfo(st, players_dict, main):
    
    fullname = st.a.string.strip()
    id_st = get_idplayer(fullname, players_dict)
    
    secondname = fullname.split()[0]
    firstname = fullname.split()[1]
    
    if main:
        role = 'Главный тренер'
    else:
        role = st.b.string.strip()[:-1]
    
    st_link = premier + st.a.get('href')
    st_page = requests.get(st_link)
    st_soup = BeautifulSoup(st_page.text, 'html.parser')
    
    bd = st_soup.find('div', class_='player-content').span.find_next('span').contents[2].strip()[:-3]
    bd_day, bd_month, bd_year, bd_strmonth = splitdate(bd)
    bd_date = str(bd_day) + ' ' + bd_month + ' ' + str(bd_year)
    height = np.nan
    weight = np.nan
        
    return role, fullname, id_st, firstname, secondname, st_link, bd_date, bd_strmonth, height, weight

In [64]:
columns = ['чемпионат', 'сезон', 'турнир', 'тур', 'месяц матча', 'матч', 'хозяева', 'хозяева голы', 'гости голы',
           'гости', 'гости\\хозяева', 'результат', 'id_матча', 'id_результата', 'амплуа', 'номер игрока', 'фио',
           'id_фио', 'имя', 'фамилия', 'минута выхода', 'минута ухода', 'минут на поле', 'гол', 'ассист','жк', 'кк',
           'пенальти', 'автогол', 'ссылка игрока', 'дата рождения', 'месяц рождения', 'рост', 'вес', 'старт\\запас',
           'команда', 'id_команды', 'CategoryColumn', 'судья']

In [65]:
def parse(season, driver=None, df_old=None):

    if driver is None:
        driver = webdriver.Firefox()
        driver_close = True
    else:
        driver_close = False
        
    try:
        premier = 'https://premierliga.ru'
        driver.get(premier + '/tournaments/youthchampionship/calendar')
        driver.find_element_by_link_text(season).click()
        time.sleep(2)
        driver.find_elements_by_class_name('tour')[-1].click()
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        if df_old is not None:
            df_old = df_old[df_old['сезон'] == season]
        
        df = pd.DataFrame(columns=columns)

        championship = 'Молодежка'
        
        if df_old is not None:
            id_match = df_old['id_матча'].unique().shape[0] + 1
        else:
            id_match = 1

        teams_dict = {'COUNTER': 0}
        players_dict = {'COUNTER': 0}

        matches = soup('tr', class_='matchtr')

        for match in tqdm(matches):
            if season == '2019/2020' and match.parent.parent.get('data-group') is not None:
                continue

            td = match.find('td', colspan='4')
            if td is not None:
                tour = int(re.search('\d+', td.p.string.strip())[0])
                continue

            if match.find('td', class_='dark-blue').a.string == '-:-':
                continue
                
            teams_name = match.p.a.string.split(' - ')
            team1_name = teams_name[0]
            team2_name = teams_name[1]
            td = match.find('td', class_='dark-blue')
            score = td.a.string
            match_name = team1_name + ' ' + score + ' ' + team2_name
            
            try:
                tournament = championship + ' ' + \
                             match.parent.parent.get('data-group')[0]
            except:
                tournament = championship
                
            if df_old is not None and (tournament + match_name == df_old['турнир'] + df_old['матч']).any():
                continue

            n = 0
            df_match = pd.DataFrame(columns=columns)

            score1 = int(score.split(':')[0])
            score2 = int(score.split(':')[1])
            result1, result2 = get_result(score1, score2)

            date = td.find_next('td').b.string
            _, _, _ , month = splitdate(date)
                
            place = match.find('td', class_='blue').a.string

            id_team1 = get_idteam(team1_name, teams_dict)
            id_team2 = get_idteam(team2_name, teams_dict)

            match_link = premier + match.p.a.get('href')
            match_page = requests.get(match_link)
            match_soup = BeautifulSoup(match_page.text)

            referee_name = match_soup.find('td', class_='official').a.contents[0][:-1]

            try:
                tp = match_soup.find('td', class_='text').p.string.strip()
            except:
                tp = None
            if tp == 'Техническое поражение':
                df_match.loc[0] = [championship, season, tournament, tour, month, match_name, team1_name, score1, score2,
                                   team2_name, 'Хозяева', result1, id_match, 2*id_match - 1] + ['ТП', np.nan, 'ТП', 0] + \
                                    [np.nan] * 17 + [team1_name, id_team1, match_link, referee_name]
                df_match.loc[1] = [championship, season, tournament, tour, month, match_name, team1_name, score1, score2,
                                   team2_name, 'Гости', result2, id_match, 2*id_match] + ['ТП', np.nan, 'ТП', 0] + \
                                    [np.nan] * 17 + [team2_name, id_team2, match_link, referee_name]
                df = df.append(df_match, ignore_index=True) 
                id_match += 1
                continue

            chrono_dict = get_chrono(match_soup)

            mainstuff = match_soup.find('table', class_='staff-club')
            reservestuff = mainstuff.find_next('table', class_='staff-club')
            for stuff in (mainstuff, reservestuff):
                main = 'Старт' if stuff is mainstuff else 'Запас'
                teams_players = stuff('tr')
                team1_players = [player.find('td', class_='text').p for player in teams_players]
                for player in team1_players:

                    if player.a is None:
                        continue

                    role, number, fullname, id_player, firstname, secondname, starttime, endtime, playtime, goals, \
                    assists, yc, rc, penalties, autogoals, player_link, bd_date, bd_month, height, weight, main = \
                    get_playerinfo(player, main, players_dict, chrono_dict)

                    df_match.loc[n] = [championship, season, tournament, tour, month, match_name, team1_name, score1, score2,
                                       team2_name, 'Хозяева', result1, id_match, 2*id_match - 1, role, number, fullname,
                                       id_player, firstname, secondname, starttime, endtime, playtime, goals, assists, yc, rc,
                                       penalties, autogoals, player_link, bd_date, bd_month, height, weight, main, team1_name,
                                       id_team1, match_link, referee_name]
                    n += 1

                team2_players = [player.find_next('td', class_='text').p for player in team1_players]
                for player in team2_players:

                    if player.a is None:
                        continue

                    role, number, fullname, id_player, firstname, secondname, starttime, endtime, playtime, goals, \
                    assists, yc, rc, penalties, autogoals, player_link, bd_date, bd_month, height, weight, main = \
                    get_playerinfo(player, main, players_dict, chrono_dict)

                    df_match.loc[n] = [championship, season, tournament, tour, month, match_name, team1_name, score1, score2,
                                       team2_name, 'Гости', result2, id_match, 2*id_match, role, number, fullname,
                                       id_player, firstname, secondname, starttime, endtime, playtime, goals, assists, yc, rc,
                                       penalties, autogoals, player_link, bd_date, bd_month, height, weight, main, team2_name,
                                       id_team2, match_link, referee_name]
                    n += 1

            if reservestuff.find_next('div', class_='title-block').find_next('div', class_='title-block').string.strip() == 'Официальные лица матча':
                teams_sts = reservestuff.find_next('table', class_='staff-club')('tr')
                main = True
            else:
                teams_sts = reservestuff.find_next('table', class_='staff-club').find_next('table', class_='staff-club')('tr')
                main = False
            team1_sts = []
            team2_sts = []
            for sts in teams_sts:
                div = sts.find('td', class_='divide')
                team1_st = div.find_previous_sibling('td', class_='coatch')
                if team1_st:
                    team1_sts.append(team1_st)
                team2_st = div.find_next_sibling('td', class_='coatch')
                if team2_st:
                    team2_sts.append(team2_st)

            for st in team1_sts:

                role, fullname, id_st, firstname, secondname, st_link, bd_date, bd_month, height, weight = get_stinfo(st, players_dict, main)
                df_match.loc[n] = [championship, season, tournament, tour, month, match_name, team1_name, score1, score2,
                                   team2_name, 'Хозяева', result1, id_match, 2*id_match - 1, role, np.nan, fullname, id_st,
                                    firstname, secondname] + [np.nan] * 9 + [st_link, bd_date, bd_month, height, weight,
                                            'Штаб', team1_name, id_team1, match_link, referee_name]
                n += 1

            for st in team2_sts:

                role, fullname, id_st, firstname, secondname, st_link, bd_date, bd_month, height, weight = get_stinfo(st, players_dict, main)
                df_match.loc[n] = [championship, season, tournament, tour, month, match_name, team1_name, score1, score2,
                                   team2_name, 'Гости', result2, id_match, 2*id_match, role, np.nan, fullname, id_st,
                                    firstname, secondname] + [np.nan] * 9 + [st_link, bd_date, bd_month, height, weight,
                                            'Штаб', team2_name, id_team2, match_link, referee_name]
                n += 1

            df = df.append(df_match, ignore_index=True)
            id_match += 1

            #if n > 2:
            #    break

        df['гол'] += df['пенальти']

        for _, player_auto in df[df['автогол'] >= 1].iterrows():
            autogoals = player_auto['автогол']
            id_result = player_auto['id_результата']
            if id_result % 2 == 0:
                id_result -= 1
            else:
                id_result += 1

            player_antiauto = df.loc[(df['id_результата'] == id_result) & (df['амплуа'] == 'Нападающий')].iloc[0].name
            df.at[player_antiauto, 'гол'] += 1

        if driver_close == True:
            driver.close()
            
        return df

    except ConnectionError:
        print('ConnectionError')
        for _, player_auto in df[df['автогол'] >= 1].iterrows():
            autogoals = player_auto['автогол']
            id_result = player_auto['id_результата']
            if id_result % 2 == 0:
                id_result -= 1
            else:
                id_result += 1

            player_antiauto = df.loc[(df['id_результата'] == id_result) & (df['амплуа'] == 'Нападающий')].iloc[0].name
            df.at[player_antiauto, 'гол'] += 1
        if driver_close == True:
            driver.close()
        return df

In [68]:
premier = 'https://premierliga.ru'
seasons = ['2019/2020', '2020/2021']

In [5]:
driver = webdriver.Firefox()

In [69]:
if 'premier.xlsx' in os.listdir():
    df_old = pd.read_excel('premier.xlsx').iloc[:, 1:]
    df = df_old.copy()
else:
    df_old = None
    df = pd.DataFrame(columns=columns)
    
for season in seasons:
    df = df.append(parse(season, driver=driver, df_old=df_old), ignore_index=True)

100%|████████████████████████████████████████████████████████████████████████████████| 216/216 [31:53<00:00,  8.86s/it]


In [70]:
col = df['матч'] + df['турнир'] + df['тур'].astype(str)
df['id_матча'] = col.replace({x: id_x + 1 for id_x, x in enumerate(col.unique())})

col = df['id_матча'].astype(str) + df['команда']
df['id_результата'] = col.replace({x: id_x + 1 for id_x, x in enumerate(col.unique())})
    
df['id_фио'] = df['фио'].replace({name: id_name + 1 for id_name, name in enumerate(df['фио'].unique())})

col = df['команда'] + df['турнир']
df['id_команды'] = col.replace({x: id_x + 1 for id_x, x in enumerate(col.unique())})

In [71]:
dd = pd.read_excel('positions.xlsx')

df['ФИО'] = df['имя'] + ' ' + df['фамилия']

cols = ['чемпионат', 'сезон', 'турнир', 'тур', 'месяц матча', 'матч', 'хозяева', 'хозяева голы', 'гости голы',
        'гости', 'гости\\хозяева', 'результат', 'id_матча', 'id_результата', 'амплуа', 'Позиция осн.', 'Позиция доп.',
        'номер игрока', 'фио', 'id_фио', 'имя', 'фамилия', 'минута выхода', 'минута ухода', 'минут на поле',
        'гол', 'ассист','жк', 'кк', 'пенальти', 'автогол', 'ссылка игрока', 'дата рождения', 'месяц рождения',
        'рост', 'вес', 'старт\\запас', 'команда', 'id_команды', 'CategoryColumn', 'судья']

df = df.merge(dd, on='ФИО').replace({'(пусто)': np.nan}).drop(columns='ФИО')[cols]

In [78]:
df.to_excel('premier.xlsx')

In [79]:
driver.close()